In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

In [2]:
# path = '/home/kai/data/jiangning/Chatbot_1/Chatbot1.0/data/'
path = '../../data/SetDueDay/'
data = pd.read_csv(path + 'cleaned_mock_up_data.csv', encoding='utf8')
print(data.shape)
data.head(10)

(12810, 2)


,label,split_text
0,2,什么 问题 ？
1,2,数额 太高 了 我 想到 了 这个 方法
2,1,我 在 港区
3,0,现在 你 可以
4,2,让 我 看看
5,2,地震
6,2,这 一次 说 下 一个 高速度 在 说话 。
7,1,筹集资金
8,2,你 说 什么 ？
9,0,三天


In [3]:
data.split_text.iloc[1]

'数额 太高 了   我 想到 了 这个 方法'

In [4]:
phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

print('fitting phrase')
phrase_vectorizer.fit(data.split_text)

print('transform phrase')
phrase = phrase_vectorizer.transform(data.split_text)

phrase

fitting phrase
transform phrase


<12810x18128 sparse matrix of type '<class 'numpy.float64'>'
	with 94566 stored elements in Compressed Sparse Row format>

# SVC

In [5]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC

l_svc = LinearSVC()
clf = CalibratedClassifierCV(l_svc) 
clf.fit(phrase, data.label)
print(clf.score(phrase, data.label))

0.915612802498


In [6]:
print(clf.predict(phrase))

[0 1 1 ..., 0 0 2]


# Logistic

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
log_r = LogisticRegression()
log_r.fit(phrase, data.label)
print(log_r.score(phrase, data.label))

0.88149882904


In [9]:
print(log_r.predict(phrase))

[0 1 1 ..., 0 0 2]


# LightGBM

In [10]:
import lightgbm as lgb

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(data.label.values)
onelabels = le.transform(data.label.values)

In [12]:
onelabels

array([2, 2, 1, ..., 0, 0, 0])

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
multicoder = MultiLabelBinarizer()
lables = multicoder.fit_transform([data.label.values])

In [14]:
lables

array([[1, 1, 1]])

In [15]:
data.label.values

array([2, 2, 1, ..., 0, 0, 0])

In [16]:
params = {
    'learning_rate': 0.2,
    'num_iterations':1000,
    'application': 'multiclassova',
    'num_class': 3,
    'num_leaves': 31,
    'verbosity': -1,
    'metric': 'multi_error',
    'data_random_seed': 2,
#     'bagging_fraction': 0.8,
#     'feature_fraction': 0.6,
    'nthread': 4,
    'lambda_l1': 1,
    'lambda_l2': 1,
    'early_stopping_round':200
} 

# lgbm_train = lgb.Dataset(phrase, data.label)
lgbm_train = lgb.Dataset(phrase, onelabels)
lgbm_val = lgb.Dataset(phrase, onelabels)
lgbm_model = lgb.train(params,lgbm_train,valid_sets=lgbm_val, verbose_eval=5)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 200 rounds.
[5]	valid_0's multi_error: 0.26815
[10]	valid_0's multi_error: 0.252849
[15]	valid_0's multi_error: 0.240125
[20]	valid_0's multi_error: 0.231928
[25]	valid_0's multi_error: 0.223966
[30]	valid_0's multi_error: 0.218189
[35]	valid_0's multi_error: 0.212881
[40]	valid_0's multi_error: 0.206401
[45]	valid_0's multi_error: 0.198439
[50]	valid_0's multi_error: 0.193677
[55]	valid_0's multi_error: 0.188525
[60]	valid_0's multi_error: 0.185246
[65]	valid_0's multi_error: 0.182436
[70]	valid_0's multi_error: 0.180328
[75]	valid_0's multi_error: 0.179235
[80]	valid_0's multi_error: 0.177283
[85]	valid_0's multi_error: 0.174317
[90]	valid_0's multi_error: 0.171975
[95]	valid_0's multi_error: 0.170648
[100]	valid_0's multi_error: 0.168384
[105]	valid_0's multi_error: 0.167057
[110]	valid_0's multi_error: 0.164793
[115]	valid_0's multi_error: 0.1637
[120]	valid_0's multi_error: 0.162451
[125]	valid_0's multi_error: 0.161436
[130]	vali

In [17]:
# print(lgbm_model.predict(phrase))

# Save

In [18]:
import pickle

# save tfidf
pickle.dump(phrase_vectorizer, open("../../savedModel/SetDueDay/tfidf.pickle", "wb"))
# pickle.dump(train_comment_features, open("train_comment_features.pickle", "wb"))
# pickle.dump(test_comment_features, open("test_comment_features.pickle", "wb"))

# save linear svc
pickle.dump(clf, open("../../savedModel/SetDueDay/LinearSVC.pickle", "wb"))
# save logistic
pickle.dump(log_r, open("../../savedModel/SetDueDay/Logistic.pickle", "wb"))
# save lightGBM
pickle.dump(lgbm_model, open("../../savedModel/SetDueDay/Lgbm.pickle", "wb"))

# test

In [18]:
test = phrase_vectorizer.transform(['滚'])
# test = phrase_vectorizer.transform(['我 在 洗澡'])

In [19]:
clf.predict_proba(test) # linear svc

array([[ 0.88240641,  0.05556803,  0.06202556]])

In [20]:
log_r.predict_proba(test) # logistic

array([[ 0.5453498 ,  0.22986951,  0.2247807 ]])

In [21]:
lgbm_model.predict(test) # light gbm

array([[ 0.37687097,  0.21631133,  0.34972291]])

# Labeling

In [27]:
# basic logic: find the max probability of 3 models, if it is larger than threshold, return the corresponding label, otherwise, return 2 (others).
result = np.vstack((clf.predict_proba(test),log_r.predict_proba(test),lgbm_model.predict(test)))
pos = np.where(result == np.max(result))

threshold = 0.7
if np.max(result)<threshold:
    label = 2
else:
    label = pos[1]
    label = label[0]
    
print('label=',label)
print('prob=',np.max(result))

label= 1
prob= 0.983824511806


# Save the whole thing

In [28]:
# import jieba
# import numpy as np

# class SetDueDay:
    
#     def __init__(self, **model):
#         """
#         suggested parameters:
#         svc, logistic, lightgbm, jieba_path,tfidf
#         """
#         self._load_model(**model)
        
#     def _load_model(self,**model):
#         self.svc = model.get('svc')
#         self.logistic = model.get('logistic')
#         self.lightgbm = model.get('lightgbm')
#         self.tfidf = model.get('tfidf')
#         # load jieba
#         jieba_path = model.get('jieba_path')
#         if jieba_path is not None:
#             jieba.load_userdict(jieba_path)
        
        
#     def classify(self, sentence):
#         sentence = jieba.cut(sentence, cut_all = False)
#         sentence = ' '.join(sentence)
#         matrix = self.tfidf.transform([sentence])
#         result = np.vstack((self.svc.predict_proba(matrix),
#                             self.logistic.predict_proba(matrix),
#                             self.lightgbm.predict(matrix)))
#         max_pred = np.max(result, axis=0)
#         max_arg = np.argmax(max_pred)
#         threshold = 0.6
#         if np.max(max_pred)<threshold:
#             label = 2
#         else:
#             label = max_arg
#         return (label, np.max(max_pred))
     
        

In [29]:
%load_ext autoreload
%autoreload 2

In [1]:
import pickle
clf = pickle.load(open("../../savedModel/SetDueDay/LinearSVC.pickle", "rb"))
lgbm_model = pickle.load(open("../../savedModel/SetDueDay/Lgbm.pickle", "rb"))
phrase_vectorizer = pickle.load(open("../../savedModel/SetDueDay/tfidf.pickle", "rb"))
log_r = pickle.load(open("../../savedModel/SetDueDay/Logistic.pickle", "rb"))

In [2]:
from SetDueDay_py import SetDueDay
ifk = SetDueDay(svc=clf, logistic=log_r, lightgbm=lgbm_model, tfidf=phrase_vectorizer,) #jieba_path='../WordCut/userdict.txt')

pickle.dump(ifk, open("../../savedModel/SetDueDay/SetDueDay.pickle", "wb"))